## Задание №1

### Описание данных

Процесс продажи устроен следующим образом:
<ul>
<li>создаётся задание на звонок человеку;</li>
<li>оператор берёт задание и пытается дозвониться до человека; если не дозванивается, мы несём некоторые затраты на время оператора и поддержку инфраструктуры;</li>
<li>если дозванивается, пытается продать наш продукт;</li>
<li>если продал, мы имеем некоторый заработок (PV) и повышенные расходы (в том числе на вознаграждение оператора).</li>

### Описание колонок в исторических данных.

<ul>
<li>ID - номер задания на звонок
<li>Флаг дозвона - 1 если в рамках задания мы дозвонились человеку, иначе 0
<li>Флаг продажи - 1 если в рамках задания мы продали наш продукт человеку, иначе 0
<li>Расходы - сумма затрат на задание
<li>PV - 0 если мы не продали наш продукт, иначе = PV проданного продукта
<li>NPV = PV - Расходы

#### Контекст
Мы много платим оператору из-за того,
что он сам по каждому заданию
звонит клиенту и ждёт ответа клиента.
Мы хотим попробовать снизить затраты на операторов,
внедрив систему,
которая будет сама звонить человеку
и в случае успеха перенаправлять задание оператору.
        
#### Тест
Мы проводим тест,
в рамках которого мы разбиваем весь поток
на две части равновероятно,
где одной части (контролю) мы продаём 
без автоматической системы дозвона,
а второй части (тесту) - с ней.
Целью этого теста
является снижение расходов на задание.

#### Статистические вводные
Уровень значимости 5%.
Мы хотели бы различать 5%-ное 
изменение целевой метрики
с вероятностью 90%.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

from statsmodels.stats.proportion import proportions_ztest

In [2]:
df = pd.read_csv('hist_telesales.csv').set_index('ID')
df

,Флаг дозвона,Флаг продажи,Расходы,PV,NPV
ID,,,,,
0,1,0,90,0,-90
1,0,0,5,0,-5
2,0,0,68,0,-68
3,1,0,22,0,-22
4,1,0,22,0,-22
...,...,...,...,...,...
72156,1,1,577,1346,769
72157,0,0,8,0,-8
72158,0,0,23,0,-23


In [3]:
df.NPV.mean() * 0.05

14.07299025789554

In [4]:
np.var(df.NPV), df.NPV.std(ddof=0)**2

(459223.3634465274, 459223.36344652745)

In [5]:
npv = sorted(df.NPV)
print(np.var(npv), np.mean(npv)*0.05)

459223.3634465273 14.07299025789554


In [6]:
stats.zscore(npv)[int(len(npv) * 0.8)]

0.868488637945507

In [7]:
stats.norm.ppf(1 - .05) - stats.norm.ppf(0.9)

0.3633020614068718

In [8]:
n = (2*np.var(npv)*(stats.norm.ppf(1 - .05) - stats.norm.ppf(0.9))**2) / (np.mean(npv)*0.05)**2

In [9]:
(2*np.var(df.NPV)*(stats.norm.ppf(1 - .05) - stats.norm.ppf(0.1))**2) / (np.mean(df.NPV)*0.05)**2

39714.59281551447

In [10]:
metr = df['Расходы']
print(np.var(metr), np.mean(metr)*0.05)

132311.345801579 10.02907734094594


In [11]:
(2*np.var(metr)*(stats.norm.ppf(1 - .05/2) - stats.norm.ppf(0.1))**2) / (np.mean(metr)*0.05)**2

27644.02902391896

In [12]:
control = pd.read_csv('Контроль.csv')
test = pd.read_csv('Тест.csv')

control1 = pd.read_csv('Контроль1.csv')
test1 = pd.read_csv('Тест1.csv')

control1['Расходы']

0          79
1          13
2          13
3          27
4        1385
         ... 
22526     224
22527      24
22528      38
22529      53
22530     327
Name: Расходы, Length: 22531, dtype: int64

In [13]:
(np.mean(test['Расходы']) - np.mean(control['Расходы'])) / np.mean(control['Расходы'])

-0.033001307328845646

In [14]:
stats.ttest_ind(test['Расходы'], control['Расходы']), stats.ttest_ind(test1['Расходы'], control1['Расходы'])

(Ttest_indResult(statistic=-2.1971751915285878, pvalue=0.028012070139204898),
 Ttest_indResult(statistic=-2.6912817609771387, pvalue=0.007120431603569667))

In [15]:
count = np.array([ sum(control1['Расходы']), sum(test1['Расходы']) ])
nobs = np.array([ len(control1['Расходы']), len(test1['Расходы']) ])

print(count, nobs, proportions_ztest(count, nobs, value=0.05, alternative='smaller'))

[4468481 4269239] [22531 22531] (nan, nan)


C:\Users\vasil\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\stats\proportion.py:866: RuntimeWarning: invalid value encountered in sqrt
  std_diff = np.sqrt(var_)


## Задание №2

### Описание данных

#### Процесс оформления кредитной карты (КК) устроен следующим образом: <br>

<li>человек заходит на сайт и заполняет информацию о себе (в том числе возраст и доход клиента) для получения кредитной карты;
<li>человек проходит скоринг (в том числе в рамках которого оценивается вероятность банкротства клиента);
<li>создаётся заявка на оформление кредитной карты;
<li>человеку выпускается виртуальная карта, а также предлагается наша услуга к этому счёту;
<li>с некоторой вероятностью человек утилизирует счёт (начинает им пользоваться);
<li>если человек не утилизирует счёт, мы несём некоторые затраты на время оператора и поддержку инфраструктуры;
<li>если человек утилизировал карту, мы имеем некоторый заработок (PV) и повышенные расходы.


### Описание колонок в исторических данных.<br>

<li>ID - номер заявки, прошедшей скоринг
<li>Возраст - возраст клиента, указанный при оформлении КК
<li>Доход клиента - доход клиента, указанный при оформлении КК
<li>Вероятность банкротства - оценка вероятности банкротства клиента, полученная с помощью скоринга по параметрам клиента
<li>Флаг утилизация счёта - совершение первой расходной операции на счёте КК
<li>Расходы - сумма затрат на заявку
<li>PV КК - 0 если счёт не утилизирован, иначе = PV кредитной карты
<li>PV услуги - 0 если счёт не утилизирован, иначе = PV этой услуги
<li>NPV = PV КК + PV услуги - Расходы

#### Контекст
Мы провели конкурентный анализ
и поняли,
что тариф нашей услуги достаточно низкий.
Мы хотим попробовать повысить тариф,
и надеемся,
что снижение конверсии в подключение услуги
компенсируется ростом дохода с этой услуги.
        
#### Тест
Мы проводим тест,
в рамках которого мы разбиваем весь поток
на две части рановероятно,
где одной части (контролю) 
мы будем предлагать старый тариф услуги,
а второй части (тесту) - новый тариф услуги.
Целью этого теста
является увеличение доходности услуги.

#### Статистические вводные
Проверка однородности выбранного параметра
осуществяется с уровнем значимости 2%.

Уровень значимости 5%.
Мы хотели бы различать 8%-ное
изменение целевой метрики
с вероятностью 90%.

In [16]:
df2 = pd.read_csv('hist_credit_card.csv')
df2

,ID,Возраст,Доход клиента,Вероятность банкротства,Флаг утилизации счёта,Расходы,PV КК,PV услуги,NPV
0,0,19,21620.835463,0.138061,0,102,0,0,-102
1,1,27,24897.990863,0.035508,1,409,11686,1754,13031
2,2,50,23989.526947,0.098793,0,16,0,0,-16
3,3,18,38442.409756,0.365661,1,788,13738,1578,14528
4,4,24,21291.521612,0.036909,1,1048,6594,2213,7759
...,...,...,...,...,...,...,...,...,...
123250,123250,32,26099.633927,0.110756,0,47,0,0,-47
123251,123251,20,24579.749275,0.113920,1,594,14268,1672,15346
123252,123252,36,34062.902531,0.247122,1,77,5950,2017,7890
123253,123253,67,24609.838522,0.020752,1,279,6278,1847,7846


In [17]:
print(np.var(df2['PV услуги']),
      np.mean(df2['PV услуги']),
      np.mean(df2['PV услуги'])*0.08,
      (stats.norm.ppf(.95) - stats.norm.ppf(0.1))**2)

1841531.021569498 1486.1052533365787 118.8884202669263 8.563847350667974


In [18]:
(2*np.var(df2['PV услуги'])*(stats.norm.ppf(.95) - stats.norm.ppf(0.1))**2) / (np.mean(df2['PV услуги'])*0.08)**2

2231.510010977145

In [19]:
print(df2['PV услуги'].corr(df2['PV КК']), 
      df2['PV услуги'].corr(df2['Расходы']),
      df2['PV услуги'].corr(df2['Флаг утилизации счёта']),
      df2['PV услуги'].corr(df2['Вероятность банкротства']),
      df2['PV услуги'].corr(df2['Доход клиента']), sep='\n')

0.12874195163991214
0.29453831012164267
0.6465244434376446
-0.13912765497669996
-0.0019369636685682623


In [20]:
controlc = pd.read_csv('Контроль_кк1.csv')
testc = pd.read_csv('Тест_кк1.csv')

testc

,ID,Возраст,Доход клиента,Вероятность банкротства,Флаг утилизации счёта,Расходы,PV КК,PV услуги,NPV
0,0,32,20662.052079,0.203710,0,95,0,0,-95
1,1,20,25261.597707,0.118478,1,288,6523,2125,8360
2,2,22,21151.687374,0.315828,1,548,12135,1542,13129
3,3,57,23726.413105,0.226921,1,149,15027,1531,16409
4,4,41,95766.375320,0.011428,1,100,11528,1720,13148
...,...,...,...,...,...,...,...,...,...
2227,2227,25,21551.731185,0.066283,1,314,5525,1577,6788
2228,2228,54,21049.901090,0.151012,1,337,5997,2675,8335
2229,2229,26,42013.239960,0.025473,1,180,8348,2092,10260
2230,2230,21,23874.521273,0.206314,0,30,0,0,-30


In [21]:
countc = np.array([ sum(controlc['PV услуги']), sum(testc['PV услуги']) ])
nobsc = np.array([ len(controlc['PV услуги']), len(testc['PV услуги']) ])
proportions_ztest(countc, nobsc, value=0.02)

C:\Users\vasil\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\stats\proportion.py:866: RuntimeWarning: invalid value encountered in sqrt
  std_diff = np.sqrt(var_)


(nan, nan)

In [31]:
def pooled_standard_error(a, b):
    std1 = a.std(ddof=0) 
    std2 = b.std(ddof=0)
    x = std1 ** 2 / a.count()
    y = std2 ** 2 / b.count()
    return (x + y)**(1/2)

def z_test(a, b): 
    return stats.norm.cdf((a.mean() - b.mean()) / pooled_standard_error(a, b))


z_test(controlc['PV услуги'], testc['PV услуги']), z_stat(controlc['PV услуги'], testc['PV услуги'])

(0.007536001943469271, -2.4306442795304437)

In [23]:
stats.ttest_ind(controlc['PV услуги'], testc['PV услуги'])

Ttest_indResult(statistic=-2.430099719362406, pvalue=0.015133845558879396)

In [25]:
stats.anderson_ksamp([controlc['PV услуги'], testc['PV услуги']]), stats.anderson_ksamp([control1['Расходы'], test1['Расходы']]) 

C:\Users\vasil\AppData\Local\Temp\ipykernel_9752\1490863765.py:1: UserWarning: p-value floored: true value smaller than 0.001
  stats.anderson_ksamp([controlc['PV услуги'], testc['PV услуги']]), stats.anderson_ksamp([control1['Расходы'], test1['Расходы']])
C:\Users\vasil\AppData\Local\Temp\ipykernel_9752\1490863765.py:1: UserWarning: p-value capped: true value larger than 0.25
  stats.anderson_ksamp([controlc['PV услуги'], testc['PV услуги']]), stats.anderson_ksamp([control1['Расходы'], test1['Расходы']])


(Anderson_ksampResult(statistic=34.576298398849694, critical_values=array([0.325, 1.226, 1.961, 2.718, 3.752, 4.592, 6.546]), significance_level=0.001),
 Anderson_ksampResult(statistic=0.08468334933111596, critical_values=array([0.325, 1.226, 1.961, 2.718, 3.752, 4.592, 6.546]), significance_level=0.25))

In [27]:
cc = pd.read_csv('Контроль_кк2.csv')
tc = pd.read_csv('Тест_кк2.csv')
tc

,ID,Возраст,Доход клиента,Вероятность банкротства,Флаг утилизации счёта,Расходы,PV КК,PV услуги,NPV
0,0,31,36305.517883,0.015659,1,952,6612,2685,8345
1,1,21,24064.092773,0.046393,1,2250,6586,1803,6139
2,2,26,23892.855918,0.054851,0,53,0,0,-53
3,3,53,21629.641133,0.124277,1,290,9010,2476,11196
4,4,67,20973.186147,0.113243,1,162,6065,1415,7318
...,...,...,...,...,...,...,...,...,...
2245,2245,27,24662.477521,0.093409,1,127,5766,1403,7042
2246,2246,18,22120.217406,0.061927,1,1030,6474,1524,6968
2247,2247,33,29339.507951,0.020111,1,758,5357,6842,11441
2248,2248,34,47760.378296,0.035629,1,152,7637,1534,9019


In [32]:
stats.anderson_ksamp([cc['PV услуги'], tc['PV услуги']]), z_test(cc['PV услуги'], tc['PV услуги'])

C:\Users\vasil\AppData\Local\Temp\ipykernel_9752\670329337.py:1: UserWarning: p-value capped: true value larger than 0.25
  stats.anderson_ksamp([cc['PV услуги'], tc['PV услуги']]), z_test(cc['PV услуги'], tc['PV услуги'])


(Anderson_ksampResult(statistic=-1.0327251285450458, critical_values=array([0.325, 1.226, 1.961, 2.718, 3.752, 4.592, 6.546]), significance_level=0.25),
 0.7578396177465547)

In [30]:
stats.ttest_ind(cc['PV услуги'], tc['PV услуги'])

Ttest_indResult(statistic=0.6992146738023493, pvalue=0.48445403551746313)